# Goal

The goal of this demo is to demonstrate how the RandomNearIncumbentOptimizer works.

# Method

1. We create a 2-parameter single-objective synthetic function for the BayesianOptimizer to maximize.
2. We either use the pass-through model or train the initial model on a bunch of random parameters.
3. We plot:
    1. The original function
    2. The models predictions
    3. The utility function values
    
4. We create the RandomNearIncumbentTracer and subscribe to all RandomNearIncumbentOptimizer events.
5. For each call to .suggest():
    1. We capture the initial incumbents and their utility values.
    2. For each call to _run_iteration():
        1. We capture the random neighbors and their utility values.
        2. We capture the new incumbents (and maybe even draw an arrow??)

6. We visualize all the data captured in 5. 

So for each scene we will need:
    1. Optionally the true objective function surface plot.
    2. Utility function surface plot.
    3. All past incumbents 3D scatter plot.
    5. All current incumbents 3D scatter plot.
    4. All current neighbors 3D scatter plot.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [20]:
import math
from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory, objective_function_config_store
from mlos.OptimizerEvaluationTools.SyntheticFunctions.EnvelopedWaves import EnvelopedWaves, enveloped_waves_config_store
from mlos.Spaces import Point


objective_function_config = Point(
    implementation=EnvelopedWaves.__name__,
    enveloped_waves_config=Point(
        num_params=2,
        num_periods=3,
        vertical_shift=0,
        phase_shift=0,
        period=2 * math.pi,
        envelope_type="linear",
        linear_envelope_config=Point(
            gradient=10
        )
    )
)
objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config)

Plotting the objective function

In [21]:
objective_function.parameter_space

  Name: domain
  Dimensions:
    x_0: [0.00, 18.85]
    x_1: [0.00, 18.85]

In [22]:
objective_function.parameter_space['x_0']

x_0: [0.00, 18.85]

In [23]:
import numpy as np
import pandas as pd

# First let's create a meshgrid of the parameters.
#
resolution_px = 100
x_0_linspace = objective_function.parameter_space['x_0'].linspace(resolution_px)
x_1_linspace = objective_function.parameter_space['x_1'].linspace(resolution_px)
meshgrids = np.meshgrid(x_0_linspace, x_1_linspace)
reshaped_meshgrids = [meshgrid.reshape(-1) for meshgrid in meshgrids]
meshgrids_dict = {
    dim_name: meshgrid
    for dim_name, meshgrid
    in zip(objective_function.parameter_space.dimension_names, reshaped_meshgrids)
}
meshgrid_params_df = pd.DataFrame(meshgrids_dict)
objectives_df = objective_function.evaluate_dataframe(meshgrid_params_df)
reshaped_objectives = objectives_df['y'].to_numpy().reshape((resolution_px, resolution_px))

In [24]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[go.Surface(
        x=x_0_linspace,
        y=x_1_linspace,
        z=reshaped_objectives
    )]
)

fig.update_layout(
    title="Objective Function",
    autosize=False,
    width=1000,
    height=1000
)

fig.show()